Because of the partial acquisitions, bbregister fails for some runs of some subjects. First shot at fixing it will be to re-run bbregister with the lyman registration outputs as an initial first guess


In [1]:
from moss.mosaic import Mosaic
import nibabel as nib
import multiprocessing
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv
import shutil
import gc

In [3]:
#preliminary housekeeping
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'subjects.txt'
sub_list = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)

In [4]:
#re-arrange directory structure
exp = 'sim_4mm'
coreg_files = ['func2anat.png','func2anat_tkreg.dat','func2anat_flirt.mat']
for sub in sub_list:
    for run in range(1,4):
        sub_path = home_dir + '/analysis/' + exp + '/' + sub + '/preproc/run_' + str(run) + '/'
        orig_coreg = sub_path + 'orig_coreg/'

        if os.path.exists(sub_path) and not os.path.exists(orig_coreg):
            os.mkdir(orig_coreg)

            for f in coreg_files:
                old = sub_path + f
                new = orig_coreg + f
                os.rename(old,new)

In [5]:
def rerun_coreg(in_tuple):
    sub, exp, run = in_tuple
    sub_path = home_dir + '/analysis/' + exp + '/' + sub + '/preproc/run_' + str(run) + '/'

    if os.path.exists(sub_path):    
        #skull strip mean_func
        mov_file = sub_path + 'mean_func.nii.gz' 
        new_mov_file = sub_path + 'mean_func_bet.nii.gz' 
#         os.remove(new_mov_file)
        cmd = ['bet2',mov_file,new_mov_file,'-f','.05']
        cmd = ' '.join(cmd)
        if not os.path.exists(new_mov_file):
            os.system(cmd)
        
        #run bbregister
        fs_dir = home_dir +'/data/' + sub
        init_reg = sub_path +  'orig_coreg/func2anat_tkreg.dat'
        out_reg = sub_path + 'func2anat_tkreg.dat'
        out_reg_fsl = sub_path + 'func2anat_flirt.mat'
        out_coreg = sub_path + '/coreg_mean_func.nii.gz'
        cmd = ['bbregister','--mov',new_mov_file,'--s',
               fs_dir,'--bold','--init-reg', init_reg, 
               '--reg',out_reg, '--fslmat',out_reg_fsl,
               '--o',out_coreg]
        cmd = ' '.join(cmd)
        if not os.path.exists(out_coreg):
            os.system(cmd)

        #save output
        out_png = sub_path + "func2anat.png"
        wm_file = fs_dir + '/mri/wm.mgz'
        wm_data = nib.load(wm_file).get_data().astype(bool).astype(int)
        m = Mosaic(out_coreg, wm_data, step=3)
        m.plot_contours(["#DD2222"])
        m.savefig(out_png)
        m.close()

In [6]:
in_tuples = []
for exp in ['sim_4mm']:
    for sub in sub_list:
        for run in range(1,4):
            in_tuples.append((sub,exp,run))

In [7]:
pool = multiprocessing.Pool(processes = 20)
output = pool.map(rerun_coreg,in_tuples)
pool.terminate()
pool.join()